<a href="https://colab.research.google.com/github/asdfasdf0311/Kswproject/blob/main/intersection_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요한 모듈가져옴
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyproj import Transformer
import pyproj
from shapely.geometry import Point as point
import geopandas as gpd
import folium
from folium import Marker
from folium.plugins import MarkerCluster

In [ ]:
from google.colab import drive #구글 드라이브에 데이터를 저장해서 데이터를 불러왔다.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
map = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=14.5)

Gateway 위치
40.4277765, -86.9111949

In [ ]:
Marker(location = [40.4277765, -86.9111949],
               popup = "Gateway").add_to(map)

데이터가져오기

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/KSWProject/data/intersection_all.xlsx')

In [ ]:
df['AH_success'] = df['AH_success'].apply(lambda x: int(x) if pd.notnull(x) and isinstance(x, float) else x) #nan제외, float -> int
df['LoRa_success'] = df['LoRa_success'].apply(lambda x: int(x) if pd.notnull(x) and isinstance(x, float) else x)

In [ ]:
df.head() #상위 5개만 보여주는 거임 - 데이터 형태만 확인

,point_num,x,y,distance,intersection,AH_success,LoRa_success,LOS,AH_obstruction(%),LoRa_obstruction(%)
0,point 1,40.427972,-86.910382,70,1,1,1,1.0,0.0,0.0
1,point 2,40.426823,-86.911975,120,0,1,1,1.0,0.0,0.0
2,point 3,40.428806,-86.912642,170,0,1,1,0.0,1.0,1.0
3,point 4,40.427869,-86.913639,210,0,1,1,0.0,1.0,1.0
4,point 5,40.429775,-86.912355,240,1,1,1,0.0,1.0,0.7


# **Data**

In [ ]:
df #row0 -> point 1, row1 -> point 2 ...etc
#2는 실험을 진행하지 않았다는 표시.

,point_num,x,y,distance,intersection,AH_success,LoRa_success,LOS,AH_obstruction(%),LoRa_obstruction(%)
0,point 1,40.427972,-86.910382,70,1,1,1,1.0,0.00,0.00
1,point 2,40.426823,-86.911975,120,0,1,1,1.0,0.00,0.00
2,point 3,40.428806,-86.912642,170,0,1,1,0.0,1.00,1.00
3,point 4,40.427869,-86.913639,210,0,1,1,0.0,1.00,1.00
4,point 5,40.429775,-86.912355,240,1,1,1,0.0,1.00,0.70
5,point 6,40.429848,-86.910381,250,0,1,1,0.0,1.00,1.00
6,point 7,40.427591,-86.908148,260,0,1,1,1.0,0.00,0.00
7,point 8,40.425959,-86.913785,300,0,0,2,0.0,1.00,1.00
8,point 9,40.425771,-86.908025,350,1,1,1,0.0,0.65,0.55
9,point 10,40.426650,-86.906925,400,0,1,1,0.0,0.70,0.70


In [ ]:
#핀 표시함수(intersection - 신호등 있는거)
def pin_intersection(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 25,
          color = 'green',
          fill = 'crimson'
           ).add_to(map)

#핀 표시함수(intersection 아닙니다)
def not_intersection(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 25,
          color = 'black',
          fill = 'crimson'
           ).add_to(map)


In [ ]:
for i in range(len(df)):
  if df['intersection'][i] == 1 :
    pin_intersection(df['x'][i],df['y'][i],i+1,df['distance'][i])
  else :
    not_intersection(df['x'][i],df['y'][i],i+1,df['distance'][i])

Green Marker = Gateway

신호등 O = Green Circle

신호등 X = Black Circle

### **< EXPERIMENT MAP > - intersection or not**

In [ ]:
map

### **< AH_success MAP >**

In [ ]:
AH_map = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=15.5)

In [ ]:
Marker(location = [40.4277765, -86.9111949],
               popup = "AP").add_to(AH_map)

**AH - working, notworking PIN 함수**

In [ ]:
#핀 표시함수(AH working) - blue
def AH_working(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 20,
          color = 'blue',
          fill = 'crimson'
           ).add_to(AH_map)

#핀 표시함수(AH not!! working) - red
def AH_notworking(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 20,
          color = 'red',
          fill = 'crimson'
           ).add_to(AH_map)


In [ ]:
for i in range(len(df)):
  if df['AH_success'][i] == 1 :
    AH_working(df['x'][i],df['y'][i],i+1,df['distance'][i])
  if df['AH_success'][i] == 0 :
    AH_notworking(df['x'][i],df['y'][i],i+1,df['distance'][i])

In [ ]:
AH_map

### **< LoRa_success MAP >**

In [ ]:
LoRa_map = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=13)

In [ ]:
Marker(location = [40.4277765, -86.9111949],
               popup = "Gateway").add_to(LoRa_map)

**LoRa - working, notworking PIN 함수**

In [ ]:
#핀 표시함수(LoRa working) - blue
def LoRa_working(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 100,
          color = 'blue',
          fill = 'crimson'
           ).add_to(LoRa_map)

#핀 표시함수(LoRa not!! working) - red
def LoRa_notworking(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 35,
          color = 'red',
          fill = 'crimson'
           ).add_to(LoRa_map)

In [ ]:
for i in range(len(df)):
  if df['LoRa_success'][i] == 1 :
    LoRa_working(df['x'][i],df['y'][i],i+1,df['distance'][i])
  #if df['LoRa_success'][i] == 0 :
    #LoRa_notworking(df['x'][i],df['y'][i],i+1,df['distance'][i])

In [ ]:
LoRa_map

## **Communication Range**

In [ ]:
map_range = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=13)
Marker(location = [40.4277765, -86.9111949],
               popup = "Gateway").add_to(map_range)
df_range = pd.read_excel('/content/drive/MyDrive/KSWProject/data/communicationrange.xlsx')

In [ ]:
from scipy.spatial import ConvexHull, QhullError

In [ ]:
# AH 포인트를 추가하는 함수
def AH_range(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=35,
        color='red',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range)

# LoRa 포인트를 추가하는 함수
def LoRa_range(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=100,
        color='blue',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range)

# 포인트를 저장할 리스트
ah_points = []
lora_points = []

# 데이터프레임을 반복하여 포인트를 추가합니다.
for i in range(len(df_range)):
    if df_range['network'][i] == 1:  # AH
        AH_range(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        ah_points.append([df_range['x'][i], df_range['y'][i]])
    elif df_range['network'][i] == 2:  # LoRa
        LoRa_range(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        lora_points.append([df_range['x'][i], df_range['y'][i]])

# ConvexHull을 사용하여 다각형 면적을 계산하는 함수
def calculate_area(points):
    if len(points) < 3:
        return 0
    hull = ConvexHull(points)
    return hull.volume  # 2D ConvexHull의 volume은 면적을 의미합니다.

# AH 포인트에 대해 다각형을 그리며 면적을 계산합니다.
ah_area_km2 = 0
if len(ah_points) >= 3:
    hull = ConvexHull(ah_points)
    ah_hull_points = [ah_points[i] for i in hull.vertices]
    folium.Polygon(
        locations=ah_hull_points,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2
    ).add_to(map_range)
    ah_area_degrees = calculate_area(ah_points)
    degrees_to_km = 111  # 위도/경도 1도의 대략적인 킬로미터 변환값
    ah_area_km2 = ah_area_degrees * (degrees_to_km ** 2)
    print(f'Area of AH polygon: {ah_area_km2} square kilometers')

# LoRa 포인트에 대해 다각형을 그리며 면적을 계산합니다.
lora_area_km2 = 0
if len(lora_points) >= 3:
    hull = ConvexHull(lora_points)
    lora_hull_points = [lora_points[i] for i in hull.vertices]
    folium.Polygon(
        locations=lora_hull_points,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2
    ).add_to(map_range)
    lora_area_degrees = calculate_area(lora_points)
    lora_area_km2 = lora_area_degrees * (degrees_to_km ** 2)
    print(f'Area of LoRa polygon: {lora_area_km2} square kilometers')


Area of AH polygon: 0.3644829790097793 square kilometers
Area of LoRa polygon: 57.52539350002244 square kilometers


In [ ]:
map_range

In [ ]:
# 필요한 802.11ah AP의 수를 계산합니다.
number_of_aps_needed = lora_area_km2 / ah_area_km2
print(f'Number of 802.11ah APs needed: {number_of_aps_needed:.2f}')

Number of 802.11ah APs needed: 157.83


#그냥 포인트만. 응응응

In [ ]:
map_range2 = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=13)
Marker(location = [40.4277765, -86.9111949],
               popup = "Gateway").add_to(map_range2)

# AH 포인트를 추가하는 함수
def AH_range2(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=35,
        color='red',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range2)

# LoRa 포인트를 추가하는 함수
def LoRa_range2(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=100,
        color='blue',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range2)


# 데이터프레임을 반복하여 포인트를 추가합니다.
for i in range(len(df_range)):
    if df_range['network'][i] == 1:  # AH
        AH_range2(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        ah_points.append([df_range['x'][i], df_range['y'][i]])
    elif df_range['network'][i] == 2:  # LoRa
        LoRa_range2(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        lora_points.append([df_range['x'][i], df_range['y'][i]])


In [ ]:
map_range2

### **면적말고 포인트만 나오는 맵**

In [ ]:
map_range2_1 = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=13)
Marker(location = [40.4277765, -86.9111949], popup = "Gateway").add_to(map_range2_1)

In [ ]:
map_range2_2 = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=12.5)
Marker(location = [40.4277765, -86.9111949], popup = "Gateway").add_to(map_range2_2)

In [ ]:
# AH 포인트를 추가하는 함수
def AH_range(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=45,
        color='green',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range2_1)

# LoRa 포인트를 추가하는 함수
def LoRa_range(x, y, n, m):
    folium.Circle(
        location=[x, y],
        popup="point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
        radius=300,
        color='blue',
        fill=True,
        fill_color='crimson'
    ).add_to(map_range2_2)

# 포인트를 저장할 리스트
ah_points = []
lora_points = []

# 데이터프레임을 반복하여 포인트를 추가합니다.
for i in range(len(df_range)):
    if df_range['network'][i] == 1:  # AH
        AH_range(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        ah_points.append([df_range['x'][i], df_range['y'][i]])
    elif df_range['network'][i] == 2:  # LoRa
        LoRa_range(df_range['x'][i], df_range['y'][i], i + 1, df_range['distance'][i])
        lora_points.append([df_range['x'][i], df_range['y'][i]])

In [ ]:
map_range2_2

fake map for presentation

In [ ]:
AH_fake = folium.Map(location=[40.4277765, -86.9111949], tiles="OpenStreetMap", zoom_start=15.5)

In [ ]:
Marker(location = [40.4277765, -86.9111949],
               popup = "AP").add_to(AH_fake)

In [ ]:
#fake
def fake_working(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 20,
          color = 'blue',
          fill = 'crimson'
           ).add_to(AH_fake)

#fake-not
def fake_notworking(x,y,n,m):
  folium.Circle(location = [x,y],
           popup = "point_" + str(n) + "<br>" + "Distance : " + str(m) + "m",
           radius = 20,
          color = 'red',
          fill = 'crimson'
           ).add_to(AH_fake)

In [ ]:
for i in range(len(df)):
  if df['AH_success'][i] == 1 :
    fake_working(df['x'][i],df['y'][i],i+1,df['distance'][i])
  if df['AH_success'][i] == 0 :
    fake_notworking(df['x'][i],df['y'][i],i+1,df['distance'][i])

In [ ]:
#구라 포인트
fake_notworking(40.435317,-86.915282,101,101)#40.435317, -86.915282
fake_notworking(40.431341,-86.910442,102,102)#40.431341, -86.910442
fake_notworking(40.428792,-86.905665,103,103)#40.428792, -86.905665
fake_notworking(40.431365,-86.916679,104,104)#40.431365, -86.916679
fake_notworking(40.429255,-86.916689,105,105)#40.429255, -86.916689

In [ ]:
AH_fake